# Plant Pathology Hypertuning

### Setup

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler

# import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

import timeit
import copy

from utils import utils, learner
from utils import PlantPathologyDataset as dataset

from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from ray.tune.suggest.hyperopt import HyperOptSearch
import hyperopt as hp

/home/braedon/.local/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [2]:
NUM_TRAIN = 75
NUM_VAL = 25
users = ('braedon', 'thomas', 'shangyi')
user = users[0]

# Needs to be the full path because of raytune
if user == users[1]:
    csv_file='C:\\Users\\tjtom\PycharmProjects\PlantPathology2021\\train.csv'
    root_dir='C:\\Users\\tjtom\PycharmProjects\PlantPathology2021\\train_images'
    mappings_dir='C:\\Users\\tjtom\PycharmProjects\PlantPathology2021\\labelMappings.csv'
elif user == users[0]:
    csv_file='/home/braedon/skole/cs175/PlantPathology2021/train.csv'
    root_dir='/home/braedon/skole/cs175/PlantPathology2021/train_images'
    mappings_dir='/home/braedon/skole/cs175/PlantPathology2021/labelMappings.csv'

plant_dataset = dataset.PlantPathologyDataset(csv_file=csv_file,
                                              root_dir=root_dir,
                                              mappings_dir=mappings_dir)
train_data = DataLoader(plant_dataset, batch_size=10, sampler= learner.ChunkSampler(NUM_TRAIN, 0))
validation_data = DataLoader(plant_dataset, batch_size=10, sampler=learner.ChunkSampler(NUM_VAL, NUM_TRAIN))

In [3]:
# Use GPU if available
if torch.cuda.is_available():
    print('Using GPU')
    dtype = torch.cuda.FloatTensor
    resources_per_trial = {'gpu': 1}
else:
    print('Using CPU')
    dtype = torch.FloatTensor
    resources_per_trial = {'cpu': 1}

Using GPU


### Testing resnet50

In [26]:
# Use GPU if available
if torch.cuda.is_available():
    resnet50 = models.resnet50().cuda() # pretrained=True is pretrained on ImageNet
else:
    resnet50 = models.resnet50()

loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.RMSprop(resnet50.parameters(), lr=1e-3)

Using GPU


In [19]:
%%timeit
torch.cuda.synchronize()
learner.train(resnet50, train_data, loss_fn, optimizer, dtype, print_every=5)
learner.check_accuracy(resnet50, validation_data, dtype)
torch.cuda.synchronize()

Starting epoch 1 / 1
2021-05-14 15:05:02,489	WARNING worker.py:1115 -- A worker died or was killed while executing task fffffffffffffffff9492f7d4957197356fc469c01000000.
torch.Size([10, 3, 400, 267]) tensor([ 3, 11,  9,  9,  0,  3,  6,  3,  0,  3])
torch.Size([10, 3, 400, 267]) tensor([0, 9, 3, 9, 1, 9, 9, 9, 3, 9])
torch.Size([10, 3, 400, 267]) tensor([4, 6, 9, 9, 3, 9, 6, 3, 0, 9])
torch.Size([10, 3, 400, 267]) tensor([3, 1, 9, 6, 9, 3, 3, 0, 0, 4])
torch.Size([10, 3, 400, 267]) tensor([ 3,  9, 11,  1,  4,  9,  3,  1,  1,  4])
t = 5, loss = 2.4781
torch.Size([10, 3, 400, 267]) tensor([10,  9,  3,  9,  6,  1,  1,  0,  3,  1])
torch.Size([10, 3, 400, 267]) tensor([9, 9, 2, 3, 1, 9, 1, 6, 4, 9])
torch.Size([5, 3, 400, 267]) tensor([9, 6, 9, 0, 9])
Checking accuracy on validation set
Got 8 / 25 correct (32.00)
Starting epoch 1 / 1
torch.Size([10, 3, 400, 267]) tensor([ 3, 11,  9,  9,  0,  3,  6,  3,  0,  3])
torch.Size([10, 3, 400, 267]) tensor([0, 9, 3, 9, 1, 9, 9, 9, 3, 9])
torch.Size(

In [20]:
%%timeit
dtype = torch.FloatTensor
resnet50 = models.resnet50()
loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.RMSprop(resnet50.parameters(), lr=1e-3)
learner.train(resnet50, train_data, loss_fn, optimizer, dtype, print_every=5)
learner.check_accuracy(resnet50, validation_data, dtype)

Starting epoch 1 / 1
torch.Size([10, 3, 400, 267]) tensor([ 3, 11,  9,  9,  0,  3,  6,  3,  0,  3])
torch.Size([10, 3, 400, 267]) tensor([0, 9, 3, 9, 1, 9, 9, 9, 3, 9])
torch.Size([10, 3, 400, 267]) tensor([4, 6, 9, 9, 3, 9, 6, 3, 0, 9])
torch.Size([10, 3, 400, 267]) tensor([3, 1, 9, 6, 9, 3, 3, 0, 0, 4])
torch.Size([10, 3, 400, 267]) tensor([ 3,  9, 11,  1,  4,  9,  3,  1,  1,  4])
t = 5, loss = 11.4072
torch.Size([10, 3, 400, 267]) tensor([10,  9,  3,  9,  6,  1,  1,  0,  3,  1])
torch.Size([10, 3, 400, 267]) tensor([9, 9, 2, 3, 1, 9, 1, 6, 4, 9])
torch.Size([5, 3, 400, 267]) tensor([9, 6, 9, 0, 9])
Checking accuracy on validation set
Got 8 / 25 correct (32.00)
Starting epoch 1 / 1
torch.Size([10, 3, 400, 267]) tensor([ 3, 11,  9,  9,  0,  3,  6,  3,  0,  3])
torch.Size([10, 3, 400, 267]) tensor([0, 9, 3, 9, 1, 9, 9, 9, 3, 9])
torch.Size([10, 3, 400, 267]) tensor([4, 6, 9, 9, 3, 9, 6, 3, 0, 9])
torch.Size([10, 3, 400, 267]) tensor([3, 1, 9, 6, 9, 3, 3, 0, 0, 4])
torch.Size([10, 3, 40

### Hypertuning

In [5]:
# This function is called by raytune in the hyperparameter tuning. 
def train_resnet(config):
    dtype = config['dtype']
    model = models.resnet50().type(dtype)
    # train_loader, test_loader = get_data_loaders()
    loss_fn = nn.CrossEntropyLoss().type(dtype)
    # optimizer = optim.SGD(
    #     model.parameters(), lr=config["lr"])#, momentum=config["momentum"])
    optimizer = optim.RMSprop(model.parameters(), config['lr'], config['alpha'], config['eps'], config['weight_decay'], config['momentum'], config['centered'])
    for i in range(10):
        learner.train(model, train_data, loss_fn, optimizer, dtype, print_every=5)
        acc = learner.check_accuracy(model, validation_data, dtype)
        tune.report(mean_accuracy=acc)
        if i % 5 == 0:
            # This saves the model to the trial directory
            torch.save(model.state_dict(), "./model.pth")

In [18]:
# Add hyperparameters here for additional tuning
# change their values to adjust the range and selection mechanism (uniform between two points, uniform from given options, iterate through all given options, etc.)
search_space = {
    'dtype': dtype, # don't change this one, not a hyperparameter
    'lr': tune.loguniform(0.001, 10),
    'alpha': tune.uniform(.1, 1.5),
    'eps': tune.uniform(1e-20, 1e-3),
    'weight_decay': tune.uniform(1e-20, 1),
    'momentum': tune.uniform(1e-20, 1),
    'centered': tune.choice((True, False))
}

num_samples = -1 # -1 is infinite
time_budget_s = 2*60*60 # max number of seconds to run
hyperopt = HyperOptSearch()
scheduler = ASHAScheduler(grace_period=1)

# This is what does the hyperparameter tuning. Right now, it's set to call the train_resnet function, and it works toward maximizing the mean_accuracy. It can also minimize loss.
analysis = tune.run(
    train_resnet,
    num_samples=num_samples,
    scheduler=scheduler,
    config=search_space,
    search_alg=hyperopt,
    resources_per_trial=resources_per_trial,
    metric="mean_accuracy",
    mode="max",
    time_budget_s=time_budget_s)

NameError: name 'scheduler' is not defined

In [21]:
analysis.get_best_config('mean_accuracy', 'max')

{'dtype': torch.cuda.FloatTensor,
 'lr': 0.012569618368549761,
 'alpha': 0.8508139209606547,
 'eps': 0.0004857270046904545,
 'weight_decay': 0.6167323331742625,
 'momentum': 0.605770928445852,
 'centered': True}

In [22]:
import pickle
pickle.dump(analysis, open('analysis.pickle', 'wb'))

In [ ]:
# To do (analysis)...

# print("Best config: ", analysis.get_best_config(
#     metric="mean_loss", mode="min"))

# # Get a dataframe for analyzing trial results.
# df = analysis.results_df